# **Modeling and Evaluation**

## Objectives

This notebook focuses on training, optimizing, and evaluating a regression model that predicts the **sale price of houses** in Ames, Iowa. It is the core modeling step for **Business Requirement 2**, where the client wants to estimate property values based on known house attributes.

Key goals include:
- Selecting a subset of features that most strongly predict sale price
- Trying multiple regression models and choosing the most effective
- Evaluating the model using R², MAE, RMSE, and visualization
- Saving the model pipeline for deployment in the Streamlit app

## Inputs

- outputs\datasets\cleaned\TrainSetCleaned.csv
- outputs\datasets\cleaned\TestSetCleaned.csv

## Outputs

- `X_train.csv` and `X_test.csv`: Train/test sets with selected features
- `y_train.csv` and `y_test.csv`: Corresponding targets (SalePrice)
- Fitted regression pipeline (`best_regressor_pipeline.pkl`)
- Feature importance plot (`feature_importance.png`)
- Evaluation metrics and model summary


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with `os.getcwd()`

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* `os.path.dirname()` gets the parent directory
* `os.chir()` defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
print("New working directory set to:", os.getcwd())

---

# Load Data

In [ ]:
import numpy as np
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/house_prices_records.csv")
      .drop(labels=['EnclosedPorch', 'WoodDeckSF'], axis=1)
  )

print(df.shape)
df.head(5)

---

# ML Pipeline with all data

# Create ML Pipeline

We define a flexible and modular ML pipeline using `sklearn.pipeline`. This pipeline includes encoding, transformations, multicollinearity reduction, scaling, feature selection, and the model itself.

All transformation steps are based on decisions made in the Feature Engineering notebook, including:
- Ordinal encoding for 4 categorical features
- Log, power, and Yeo-Johnson transformations for skewed numeric features
- SmartCorrelatedSelection with threshold = 0.6

In [ ]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from feature_engine.imputation import ArbitraryNumberImputer, MeanMedianImputer, CategoricalImputer
from feature_engine.encoding import OrdinalEncoder
from feature_engine.transformation import LogTransformer, PowerTransformer, YeoJohnsonTransformer
from feature_engine.selection import SmartCorrelatedSelection

# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

def PipelineOptimization(model):
    num_impute_zero = ['2ndFlrSF', 'MasVnrArea', 'GarageYrBlt']
    num_impute_median = ['BedroomAbvGr', 'LotFrontage']
    cat_impute_missing = ['BsmtExposure', 'BsmtFinType1', 'GarageFinish']
    ordinal_encode = ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual']

    log_transform = ['GrLivArea']
    log10_transform = ['1stFlrSF']
    yeojohnson_transform = ['GarageArea', 'LotFrontage', '1stFlrSF']
    power_transform = ['TotalBsmtSF']

    pipeline = Pipeline([
        ("num_zero", ArbitraryNumberImputer(arbitrary_number=0, variables=num_impute_zero)),
        ("num_median", MeanMedianImputer(imputation_method='median', variables=num_impute_median)),
        ("cat_missing", CategoricalImputer(imputation_method='missing', variables=cat_impute_missing)),

        ("ordinal_encoder", OrdinalEncoder(encoding_method='arbitrary', variables=ordinal_encode)),

        ("log_transform", LogTransformer(variables=log_transform)),
        ("log10_transform", LogTransformer(variables=log10_transform, base='10')),
        ("yeojohnson", YeoJohnsonTransformer(variables=yeojohnson_transform)),
        ("power_transform", PowerTransformer(variables=power_transform)),

        ("correlation_filter", SmartCorrelatedSelection(
            method='spearman',
            threshold=0.8,
            selection_method='variance')),
        
        ("feat_scaling", StandardScaler()),

        ("feat_selection",  SelectFromModel(model)),

        ("model", model),
    ])

    return pipeline


Custom Class for hyperparameter optimisation

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches


## Split Train Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)


In [ ]:
X_train.head(10)

## Grid Search CV - Sklearn

### Use default hyperparameters to find most suitable algorithm

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

### Perform Quick Hyperparameter Search

This cell uses the `HyperparameterOptimizationSearch` class to perform a quick grid search over default hyperparameters for multiple regression models. The goal is to identify the most suitable algorithm for predicting house prices. The search evaluates models using 5-fold cross-validation and the R² scoring metric. Results will help determine which model to focus on for further optimization.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

### Do an extensive search on the most suitable model to find the best hyperparameter configuration.

Define model and parameters, for Extensive Search

In [ ]:
models_search = {
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "LinearRegression": LinearRegression(),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "LinearRegression": LinearRegression(),
}

params_search = {
    "ExtraTreesRegressor": {
        'model__n_estimators': [100, 200],
        'model__max_depth': [10, 20],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2],
    },
    "LinearRegression": {
        'model__fit_intercept': [True],
    },
    "RandomForestRegressor": {
        'model__n_estimators': [100, 200],
        'model__max_depth': [10, 20],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2],
    }
}

Extensive GridSearch CV

In [ ]:
import warnings
# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)


search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

Check the best model

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

Parameters for best model

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

Define the best regressor

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

## Assess feature importance

We visualize the top 5 most important features according to the ExtraTreesRegressor model. These are the features that contributed most to predicting house prices.

This helps:
- Understand what drives model decisions
- Align findings with domain knowledge
- Potentially drop weak features in future optimization

As expected, features like overall quality (`OverallQual`), above-ground living area (`GrLivArea`), and garage size (`GarageArea`) are key contributors.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

sns.set_style('whitegrid')

data_cleaning_feat_eng_steps = 9
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support(
)].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],
    'Importance': best_regressor_pipeline['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()


From the chart above, we observe that `OverallQual` dominates in importance, indicating that overall construction and finish quality are the strongest indicator of sale price in Ames, Iowa. This insight could also guide further feature selection or transformation in future iterations.

## Evaluate Regressor on Train and Tests Sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)


def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:', np.sqrt(
        mean_squared_error(y, prediction)).round(3))
    print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    plt.show()


Evaluate Performance

In [ ]:
regression_performance(X_train, y_train, X_test, y_test,best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test,
                            best_regressor_pipeline)

# Refit a Simpler Pipeline with Only Top Features

We rebuild the pipeline using only the most important features to evaluate performance and interpretability trade-offs. This helps simplify the model without sacrificing too much accuracy.


## Rewrite Pipeline

In [ ]:
def PipelineTop5(model):
    log_transform = ['GrLivArea']
    yeojohnson_transform = ['GarageArea',]
    power_transform = ['TotalBsmtSF']

    pipeline_selected = Pipeline([
        ("log_transform", LogTransformer(variables=log_transform)),
        ("yeojohnson", YeoJohnsonTransformer(variables=yeojohnson_transform)),
        ("power_transform", PowerTransformer(variables=power_transform)),
        
        ("feat_scaling", StandardScaler()),

        ("model", model),
    ])

    return pipeline_selected

## Split Train Test Set, only with best features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)


### Subset Best Features

We now limit the dataset to only the top 5 most important features (from the previous ExtraTrees model). This lets us test how much performance is retained with fewer predictors.

In [ ]:
X_train = X_train[best_features]
X_test = X_test[best_features]

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:", X_test.shape, y_test.shape)
X_train.head(3)

## Grid Search CV – Sklearn

We are using the same model from the previous GridCV search

In [ ]:
models_top5 = {
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0)
}

And we are gonna check for the best parameters just as in the previous GridCV search

In [ ]:
params_search['ExtraTreesRegressor']

### Define Hyperparameter Grid for Top 5 Features

This cell defines the hyperparameter grid (`params_top5`) for the `ExtraTreesRegressor` model, which was identified as the best-performing model in the previous grid search.

These hyperparameters will be used in a new grid search to optimize the model's performance when trained on the top 5 most important features. This step ensures that the model is fine-tuned for the reduced feature set.

In [ ]:
params_top5 = {
    "ExtraTreesRegressor": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [10, 20, None],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]
    }
}
params_top5

### Perform Hyperparameter Optimization for Top 5 Features

The results of the grid search will help identify the best hyperparameter configuration for the `ExtraTreesRegressor` model when trained on the reduced feature set.

In [ ]:
search = HyperparameterOptimizationSearch(models=models_top5, params=params_top5)
search.fit(X_train, y_train,
                 scoring='r2',
                 n_jobs=-1,
                 cv=5)

View Results and Select Best Pipeline

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

Check the best model

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model
best_pipeline_top5 = grid_search_pipelines[best_model].best_estimator_
best_pipeline_top5

## Assess feature importance

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

# Define the pipeline_selected using the best model
pipeline_selected = PipelineOptimization(ExtraTreesRegressor(
    max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=100, random_state=0))

# Fit the pipeline on the training data
pipeline_selected.fit(X_train, y_train)

# number of data cleaning and feature engineering steps in the pipeline
data_cleaning_feat_eng_steps = 3
columns_after_data_cleaning_feat_eng = (Pipeline(pipeline_selected.steps[:data_cleaning_feat_eng_steps])
                                        .fit(X_train, y_train)
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng,
    'Importance': pipeline_selected['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()


The importance levels are virtually unchanged in relation to each other.

## Evaluate Classifier on Train and Test Sets

In [ ]:
regression_performance(X_train, y_train, X_test, y_test,best_pipeline_top5)
regression_evaluation_plots(X_train, y_train, X_test, y_test,
                            best_pipeline_top5)

Although we dropped many features, the model performance remained stable, showing **slight increase in R² Score from 0.84 to 0.844**. This suggests the removed features were not adding significant predictive value, confirming that ExtraTrees is robust and able to focus on strong predictors.

# Save Artifacts for Deployment and Dashboard

These outputs will allow the Streamlit app to:

- Load the final trained pipeline
- Access the top 5 features
- Read input format from training data
- Visualize model insights like feature importance

## Create Output Directory

In [ ]:
import os
import joblib

version = "v1"
output_dir = f"outputs/ml_pipeline/predict_house_price/{version}"

try:
  os.makedirs(name=output_dir, exist_ok=True)
except Exception as e:
  print(e)

print(f"✅ Output directory created at: {output_dir}")


## Save Train and Test Sets

In [ ]:
# Save features
X_train.to_csv(f"{output_dir}/X_train.csv", index=False)
X_test.to_csv(f"{output_dir}/X_test.csv", index=False)

# Save targets
y_train.to_csv(f"{output_dir}/y_train.csv", index=False)
y_test.to_csv(f"{output_dir}/y_test.csv", index=False)

print("✅ Train/test sets saved.")

## Save Final Model Pipeline

In [ ]:
joblib.dump(value=best_pipeline_top5, filename=f"{output_dir}/pipeline_top5.pkl")

print("✅ Final model pipeline saved.")

## Save Feature Importance Plot

In [ ]:
# Top 5 features were already used to train the pipeline
df_feature_importance_top5 = df_feature_importance[df_feature_importance["Feature"].isin(X_train.columns)]

df_feature_importance_top5.plot(kind='bar', x='Feature', y='Importance', figsize=(8, 4))
plt.title("Top 5 Features - Feature Importance")
plt.tight_layout()
plt.savefig(f"{output_dir}/feature_importance_top5.png", bbox_inches='tight')
plt.show()

print("✅ Feature importance plot saved.")

## Save Feature List (for dashboard inputs)

In [ ]:
joblib.dump(value=list(X_train.columns), filename=f"{output_dir}/feature_list.pkl")

print("✅ Feature list saved for dashboard.")

## Save evaluation metrics to `.json`

In [ ]:
import json

metrics = {
    "R2_train": r2_score(y_train, best_pipeline_top5.predict(X_train)).round(3),
    "MAE_train": mean_absolute_error(y_train, best_pipeline_top5.predict(X_train)).round(3),
    "RMSE_train": np.sqrt(mean_squared_error(y_train, best_pipeline_top5.predict(X_train))).round(3),
    "R2_test": r2_score(y_test, best_pipeline_top5.predict(X_test)).round(3),
    "MAE_test": mean_absolute_error(y_test, best_pipeline_top5.predict(X_test)).round(3),
    "RMSE_test": np.sqrt(mean_squared_error(y_test, best_pipeline_top5.predict(X_test))).round(3)
}

# Save to JSON file
with open(f"{output_dir}/model_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)

print("✅ Model metrics saved as JSON.")
